# What is the distance between cities and their capital?
*This is a real project for one of my previous company.*

### Problem Statement:
We had a geographic limitation to operate, we should accept registration only by people who live in the capital or near.

### Solution:
Define 4 capitals and some cities (based on population) and conected to a website to extract the distance. \
After this process we use the csv file to determine the maximum distance that we should accept.

### Libs
- Requests - To conect with a site that have the information needed
- Pandas - To load and manipulate data
- Scrapy - To scrap data from a website

## Step 1 - Importing libs and load local csv

In [1]:
import pandas as pd
from scrapy import Selector
import requests

df_cities = pd.read_csv('cities.csv')

# printing the first rows to see if everythin is fine
df_cities.head()

,state,city,capital
0,MG,Belo Horizonte,Belo Horizonte
1,MG,Baldim,Belo Horizonte
2,MG,Betim,Belo Horizonte
3,MG,Brumadinho,Belo Horizonte
4,MG,Caeté,Belo Horizonte


## Step 2 - Itearate over dataframe, extract the distance and add to a new column
2.1 The querystring to determine the "from" and the "to", we get this from each line of dataframe\
2.2 The x_path to get the string from website with the distance.\
2.3 The try/ except was implemented to avoid errors.


In [2]:
for i, row in df_cities.iterrows():

    url = "http://www.distanciasentrecidades.com/pesquisa"

    querystring = {"from":row['city'],"to":row['capital']}

    response = requests.request("GET", url, params=querystring)

    x_path_distance = '//*[@id="kmslinearecta"]//text()'

    sel = Selector(text=response.text)

    try:  
        distance = sel.xpath(x_path_distance).get()

        df_cities.loc[i, 'Distance'] = distance.replace('.', ',')
    except:
        df_cities.loc[i, 'Distance'] = 'Não foi possível encontrar'
        
display(df_cities)

,state,city,capital,Distance
0,MG,Belo Horizonte,Belo Horizonte,0
1,MG,Baldim,Belo Horizonte,"69,036"
2,MG,Betim,Belo Horizonte,"23,951"
3,MG,Brumadinho,Belo Horizonte,"36,34"
4,MG,Caeté,Belo Horizonte,"30,809"
...,...,...,...,...
113,SP,São Caetano do Sul,São Paulo,"11,758"
114,SP,São Lourenço da Serra Suzano,São Paulo,"36,458"
115,SP,Suzano,São Paulo,"36,458"
116,SP,Taboão da Serra,São Paulo,"13,742"


#### Step 3 - Load de dataframe to CSV
3.1 In this case the CSV should be used by Product and Ops team do define the business rules.

In [3]:
df_cities.to_csv('distance_final.csv')